In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import pickle
from datetime import datetime, timedelta
import re

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 처음 시도한 방법

In [ ]:
# url로 접근
options = webdriver.ChromeOptions()
options.add_argument('headless') 
options.add_argument('window-size = 1920x1980')

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://news.nate.com')

driver.find_element_by_id('input_search').send_keys('아모레퍼시픽')

driver.find_element_by_id('acBtn').click()

iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
driver.switch_to_frame(iframe)

#driver.find_element_by_css_selector('dl.option-list > dd.opt02 > span.sel02').click() # 검색영역 '제목' 선택

# 기간 설정
driver.find_element_by_name('ps1').clear()
driver.find_element_by_name('ps1').send_keys('20100806')
driver.find_element_by_name('ps2').clear()
driver.find_element_by_name('ps2').send_keys('20180912')
time.sleep(2)


driver.find_element_by_class_name('type-search-btn').click()
justin = 0
while True:
    try:
        if driver.find_element_by_css_selector('div.none-result'):
            driver.refresh()
    except Exception as e:
        print(e)
        break
time.sleep(1)

lastdate = datetime.datetime.strptime('20180912', '%Y%m%d')
url_list = []
p = 20
while True:
    print('jjjjjjjjjjjjj : ', justin)
    print('While문 진입')
    try:
        #if driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe'):
        iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
        driver.switch_to_frame(iframe)
        page_url = driver.find_element_by_css_selector('div.paging > span > a').get_attribute('href')
        page_url = page_url[:-2]
        for i in range(p):
            try:
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:
                    print(i, '   이건데 왜 이거 실행함?')
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate + '   *' * 100)                
                
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if i == 99:
                    print('진행상황 ({}/99)'.format(i + justin*100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin * 100), index = False)
                    url_list = []
            except Exception as e:
                print(e)
                print('이번 페이지는 크롤링 못 했다. (%d페이지)' %(i+1))
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:
                    print(i, '   이건데 왜 이거 실행함?')                    
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate + '   *' * 100)
                    
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if i == 99:
                    print('진행상황 ({}/99)'.format(i + justin * 100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin * 100), index = False)
                    url_list = []
                
    except Exception as e:
        print(e)
        print('No iframe')
        #iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
        #driver.switch_to_frame(iframe)
        page_url = driver.find_element_by_css_selector('div.paging > span > a').get_attribute('href')
        page_url = page_url[:-2]
        for i in range(p):
            try:
                print(i,' *' * 100)
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:                
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate, '   *' * 100)                
                
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if  i == 99:
                    print('진행상황 ({}/99)'.format(i + justin * 100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin*100), index = False)
                    url_list = []
                    
            except Exception as e:
                print(i,' * ' * 100)                
                print(e)
                print('이번 페이지는 크롤링 못 했다. (%d페이지)' %(i+1))
                URL = page_url + str(1 + (i * 10))
                driver.get(URL)
                time.sleep(1)
                titles = driver.find_elements_by_css_selector('dt.text-inline')
                
                if p > 10:                
                    dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
                    dtime = [datetime.datetime.strptime(d, '%Y%m%d') for d in dates]
                    if lastdate > min(dtime):
                        lastdate = min(dtime)
                    print(lastdate, '   *' * 100)                
                
                for j in range(len(titles)):
                    url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
                if  i == 99:
                    print('진행상황 ({}/99)'.format(i + justin * 100))
                    pd.Series(url_list).to_csv('news_data/news{}.csv'.format(i + justin * 100), index = False)
                    url_list = []
        
    # 마지막 페이지의 마지막 글의 날짜
    #date_list = driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')
    #date = date_list[9].text[-16:-6].replace('.', '')
    
    dateinput = lastdate.strftime('%Y%m%d')
    driver.find_element_by_name('ps1').clear()
    driver.find_element_by_name('ps1').send_keys('20100806')
    driver.find_element_by_name('ps2').clear()
    driver.find_element_by_name('ps2').send_keys(dateinput)
    time.sleep(2)
    print(dateinput)
    #driver.find_element_by_css_selector('dl.option-list > dd.opt02 > span.sel02').click() # 검색영역 '제목' 선택
    driver.find_element_by_class_name('type-search-btn').click()
    print('endendendendenden : ', justin)
    justin = justin + 1
    if justin == 123:
        break

# 새로 도전(성공!)
- 날짜를 7일마다 변경하여 크롤링

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# url로 접근
options = webdriver.ChromeOptions()
options.add_argument('headless') 
options.add_argument('window-size = 1920x1980')

driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://news.nate.com')

driver.find_element_by_id('input_search').send_keys('아모레퍼시픽')

driver.find_element_by_id('acBtn').click()

iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
driver.switch_to_frame(iframe)

#driver.find_element_by_css_selector('dl.option-list > dd.opt02 > span.sel02').click() # 검색영역 '제목' 선택

# 기간 설정
driver.find_element_by_name('ps1').clear()
driver.find_element_by_name('ps1').send_keys('20180905')
driver.find_element_by_name('ps2').clear()
driver.find_element_by_name('ps2').send_keys('20180912')
time.sleep(2)


driver.find_element_by_class_name('type-search-btn').click()
time.sleep(5)

justin = 0
start_date = '20180912'
middle_date = '20180905'
finish_date = '20100806'
delta = -7

while True:
    try:
        if driver.find_element_by_css_selector('div.none-result'):
            driver.refresh()
            time.sleep(3)
            print('검색 클릭 후 창 출력 실패 -> refresh')
            ### 여기가 문제구먼 날짜가 바뀌는 부분
            driver.find_element_by_name('ps1').clear()
            driver.find_element_by_name('ps1').send_keys(middle_date)
            driver.find_element_by_name('ps2').clear()
            driver.find_element_by_name('ps2').send_keys(start_date)
            driver.find_element_by_class_name('type-search-btn').click()
    except Exception as e:
        print('페이지가 정상적으로 출력')
        break
time.sleep(1)

stop = 0
while True:
    try:
        iframe = driver.find_element_by_id('contentsWraper').find_element_by_tag_name('iframe')
        driver.switch_to_frame(iframe)
        break
    except Exception as e:
        print('iframe이 이미 있습니다.')
        break
    stop = stop + 1
    if stop == 10:
        break

        
        
try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.paging > span > a'))
    )
except:
    print('시작이 안 좋네')    
page_url = driver.find_element_by_css_selector('div.paging > span > a').get_attribute('href')
page_url = page_url[:-2]

i = 1
url_list = []
stop_hard = 1
justin = 1
while True:
    try:
        
        URL = page_url + str(10 * i - 9)
        driver.get(URL)
        
        # 마지막 페이지 확인 후 다시 처음으로
        try:
            element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.paging'))
            )
        except:
            print('도저히 페이지 누루는 부분을 못 찾네')
            drver.refresh()
            
        last_URL = driver.find_element_by_link_text('마지막').get_attribute('href')
            
        try:
            element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'dt.text-inline'))
            )
        except:
            print('도저히 이 element를 못 찾네 ', str(10 * i - 9))
            driver.refresh()
         
        
        titles = driver.find_elements_by_css_selector('dt.text-inline')
        dates = [day.text[-16:-6].replace('.', '') for day in driver.find_elements_by_css_selector('ul.search-list dl dd.text-inline span')]
        print(len(titles), ' ', URL)
        
        if len(titles) == 0:
            raise print('이 부분은 title이 0개', URL)
            
        for j in range(len(titles)):
            url_list.append(titles[j].find_element_by_tag_name('a').get_attribute('href').replace('https://news.nate.com/view/', ''))
            
        if True:
            print('\r{}/13000'.format(i + justin * 100 - 100), end = '')
            print('진행상황 ({}/125000)'.format(i * justin * 10), 'url 개수 = %d' %(len(url_list)))
            # pd.Series(url_list).to_csv('news_data/news{}.csv'.format(justin), index = False)
            
 
            
        i = i + 1
        
    except Exception as e:
        # print(e)
        print('%d 번 페이지는 크롤링 못 함, refresh 필요' %i)
        None

    if URL == page_url + str(int(last_URL[-3:]) + 10):
        print('끝났습니다. 정상적으로 크롤링되었습니다. %s' %middle_date)
        print('')
        
        # dates = {day for d in url_list for day in re.findall('([0-9]+)n', d)}
        # date_format = [datetime.strptime(dd, '%Y%m%d') for dd in dates]
        # date_input = min(date_format)
        
        start_date = datetime.strptime(start_date, '%Y%m%d')
        middle_date = datetime.strptime(middle_date, '%Y%m%d')
        start_date = middle_date
        middle_date = middle_date - timedelta(days = 7)                
        
        start_date = start_date.strftime('%Y%m%d')
        middle_date = middle_date.strftime('%Y%m%d')
        print(start_date, '->', middle_date)
        
        driver.find_element_by_name('ps1').clear()
        driver.find_element_by_name('ps1').send_keys(middle_date)
        driver.find_element_by_name('ps2').clear()
        driver.find_element_by_name('ps2').send_keys(start_date)
        time.sleep(0.5)
        driver.find_element_by_class_name('type-search-btn').click()
        time.sleep(5)        
        while True:
            try:
                if driver.find_element_by_css_selector('div.none-result'):
                    driver.find_element_by_name('ps1').clear()
                    driver.find_element_by_name('ps1').send_keys(finish_date)
                    driver.find_element_by_name('ps2').clear()
                    driver.find_element_by_name('ps2').send_keys(start_date)
                    time.sleep(0.5)
                    driver.find_element_by_class_name('type-search-btn').click()
                    time.sleep(3)
            except:
                print('날짜 변경 시 오류 없음')
                try:
                    element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.paging > span > a'))
                    )
                    page_url = driver.find_element_by_css_selector('div.paging > span > a').get_attribute('href')
                    page_url = page_url[:-2]
                except:
                    print('실제로 날짜는 안 바뀜')
                    None
                break
        pd.Series(url_list).to_csv('news_data/news{}.csv'.format(justin), index = False)
        url_list = []
        i = 1
        justin += 1
        
    stop_hard += 1
    if stop_hard == 1000000 | justin == 130:
        print('오류가 발생했습니다. 강제로 종류합니다.')
        break

페이지가 정상적으로 출력
iframe이 이미 있습니다.
10   https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&f3=1&ps=3&ps1=20180905&ps2=20180912&o=P&e=1
1/13000진행상황 (10/125000) url 개수 = 10
10   https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&f3=1&ps=3&ps1=20180905&ps2=20180912&o=P&e=11
2/13000진행상황 (20/125000) url 개수 = 20
10   https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&f3=1&ps=3&ps1=20180905&ps2=20180912&o=P&e=21
3/13000진행상황 (30/125000) url 개수 = 30
10   https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&f3=1&ps=3&ps1=20180905&ps2=20180912&o=P&e=31
4/13000진행상황 (40/125000) url 개수 = 40
10   https://news.search.nate.com/iframe/search.html?q=%EC%95%84%EB%AA%A8%EB%A0%88%ED%8D%BC%EC%8B%9C%ED%94%BD&f3=1&ps=3&ps1=20180905&ps2=20180912&o=P&e=41
5/13000진행상황 (50/125000) url 개수 = 50
10   https://ne

KeyboardInterrupt: 

# URL 추출 성공 후 내용 추출

In [181]:
import glob
import pandas as pd


url = 'https://news.nate.com/view/'


# all_files = glob.glob(path_input + '/*.csv')
# df_list = [pd.read_csv(file, encoding = 'utf-8', header = None) for file in all_files]
# df = pd.concat(df_list, sort = False)
# df.columns = ['URL']
df = pd.read_csv('news_url(20160928-20180912).csv', encoding = 'utf-8')

df.drop_duplicates('URL', keep = 'first', inplace = True)
url_list = list(set(df['URL'].values))
list_len = len(url_list)

driver = webdriver.Chrome('chromedriver.exe')

# 1000개씩 자르자
i = 1

title_list = []
content_list = []
date_list = []
error_URL = []


for j, u in enumerate(url_list):
    URL = url + u
    driver.get(URL)

    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.ID, "articleView"))
        )
    except:
        print('해당 URL오류 발생 -> 따로 추가했음(error_URL)')
        error_URL.append(URL)
    
    try:
        title = driver.find_element_by_class_name('articleSubecjt').text # 제목
        date = driver.find_element_by_css_selector('.firstDate em').text # 날짜
        content = driver.find_element_by_id('realArtcContents').text # 내용
        
        title_list.append(title)
        content_list.append(content)
        date_list.append(date)
        
    except:
        title = None
        date = None
        content = None
        error_URL.append(URL)
        
    if len(title_list) == 1000: # 원래는 1000
        
        data = {'title' : title_list, 'content' : content_list, 'date' : date_list}
        df = pd.DataFrame(data)
        df.to_csv('news_content{}.csv'.format(i * 1000), index = False, encoding = 'utf-8')
        print('\r 파일저장 현황 : {}/29'.format(i), end = '')
        
        title_list = []
        content_list = []
        date_list = []
    
        i += 1
        
    print('\r 진행상황 : {}/29260'.format(j + 1), end = '')

 진행상황 : 383/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 859/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 860/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 861/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 862/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 863/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 864/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 865/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 866/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 867/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 868/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 1014/29260

TimeoutException: Message: timeout
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


# 중간에 끊겼을 때!!

In [ ]:
start = 23551 # 진행상황과 같은 숫자 입력 ex) 15/29260 -> 15입력

for j, u in enumerate(url_list[start:]):
    URL = url + u
    driver.get(URL)

    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.ID, "articleView"))
        )
    except:
        print('해당 URL오류 발생 -> 따로 추가했음(error_URL)')
        error_URL.append(URL)
    
    try:
        title = driver.find_element_by_class_name('articleSubecjt').text # 제목
        date = driver.find_element_by_css_selector('.firstDate em').text # 날짜
        content = driver.find_element_by_id('realArtcContents').text # 내용
        
        title_list.append(title)
        content_list.append(content)
        date_list.append(date)
        
    except:
        title = None
        date = None
        content = None
        error_URL.append(URL)
        
    if len(title_list) == 1000: # 원래는 1000
        
        data = {'title' : title_list, 'content' : content_list, 'date' : date_list}
        df = pd.DataFrame(data)
        df.to_csv('news_content{}.csv'.format(i * 1000), index = False, encoding = 'utf-8')
        print('\r 파일저장 현황 : {}/29'.format(i), end = '')
        
        title_list = []
        content_list = []
        date_list = []
    
        i += 1
        
    print('\r 진행상황 : {}/29260'.format(start + j + 1), end = '')
    if URL == url + url_list[-1]:
        break

 진행상황 : 23575/29260

In [7]:
url_list[-1]

'20170628n08269'

In [6]:
len(title_list)

521

# 크롬창이 닫혔을 때!
- 밑에 코드 먼저 실행하고 위에 다시 실행~

In [ ]:
driver = webdriver.Chrome('chromedriver.exe')

# 특수하게 쥬피터까지 꺼졌을 때

In [2]:
url = 'https://news.nate.com/view/'


# all_files = glob.glob(path_input + '/*.csv')
# df_list = [pd.read_csv(file, encoding = 'utf-8', header = None) for file in all_files]
# df = pd.concat(df_list, sort = False)
# df.columns = ['URL']
df = pd.read_csv('news_url(20160928-20180912).csv', encoding = 'utf-8')

df.drop_duplicates('URL', keep = 'first', inplace = True)
url_list = list(set(df['URL'].values))
list_len = len(url_list)

driver = webdriver.Chrome('chromedriver.exe')

# 1000개씩 자르자
i = 1

title_list = []
content_list = []
date_list = []
error_URL = []

start = 1000
for j, u in enumerate(url_list[start:]):
    URL = url + u
    driver.get(URL)

    try:
        element = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.ID, "articleView"))
        )
    except:
        print('해당 URL오류 발생 -> 따로 추가했음(error_URL)')
        error_URL.append(URL)
    
    try:
        title = driver.find_element_by_class_name('articleSubecjt').text # 제목
        date = driver.find_element_by_css_selector('.firstDate em').text # 날짜
        content = driver.find_element_by_id('realArtcContents').text # 내용
        
        title_list.append(title)
        content_list.append(content)
        date_list.append(date)
        
    except:
        title = None
        date = None
        content = None
        error_URL.append(URL)
        
    if len(title_list) == 1000: # 원래는 1000
        
        data = {'title' : title_list, 'content' : content_list, 'date' : date_list}
        df = pd.DataFrame(data)
        df.to_csv('news_content{}.csv'.format(i * 1000), index = False, encoding = 'utf-8')
        print('\r 파일저장 현황 : {}/29'.format(i), end = '')
        
        title_list = []
        content_list = []
        date_list = []
    
        i += 1
        
    print('\r 진행상황 : {}/29260'.format(start + j + 1), end = '')

 진행상황 : 1537/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 1731/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 2244/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 3357/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 3801/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 3975/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 3981/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 4708/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 4713/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 4765/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 5889/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 6783/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 8007/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 8188/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 8276/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 8480/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 8633/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 8946/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 9000/29260해당 URL오류 발생 -> 따로 추가했음(error_URL)
 진행상황 : 9201

KeyboardInterrupt: 

# 새로 배운 사실
1. try except은 혁명이다.
2. 크롤링을 못 하도록 수법을 걸어놨는데, iframe페이지로 들어가니 상관 없어짐

# 오류
- 초반 1000개 없어짐, 다시 해야 함

In [ ]:
import glob

all_files = glob.glob('*.csv')